In [1]:
library(plyr)
library(tidyverse)
library(readxl)
library(ggpubr)

── Attaching core tidyverse packages ────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ──────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::desc()      masks plyr::desc()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::id()        masks plyr::id()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::mutate()    masks plyr::mutate()
✖ dplyr::rename()    masks plyr::rename()
✖ dplyr::summarise() masks plyr::summarise()
✖ dplyr::summarize() masks plyr::summarize()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching 

In [2]:
`%notin%` <- Negate(`%in%`)

In [3]:
runVersion = "test"

In [4]:
MSigDB_unmaskingDict <- c("Name A"="LLM", "Name B"= "GO", "Neither" = "Neither", "Tied" = "Tied")

In [5]:
NeST_unmaskingDict <- c("Name B"="LLM", "Name A"= "GO",  "Neither" = "Neither", "Tied" = "Tied")

In [6]:
## read in answers from google sheet - human input

In [7]:
getHumanConsensus = function(Winner_human1,Winner_human2,  Winner_human3, OverridingDecision = NaN ){
  # dataframe row 
  if (!is.na(OverridingDecision)){
    humanConsensus = OverridingDecision
  } else {
    winnerList =  c(Winner_human1, Winner_human2,Winner_human3 )
    sortedCounts = sort(table(winnerList), decreasing = TRUE)
    if  (sortedCounts[1] ==1){
      # all are tied
        if (("Tied" %in% winnerList) & ("Neither" %in% winnerList) ){
            humanConsensus = "Neither" # Neither is stronger than Tied
      } else if ("Tied" %in% winnerList){
        humanConsensus = "Tied"
      } else if ("Neither" %in% winnerList){
        humanConsensus = "Neither"
      } else {
        humanConsensus = "need help"
      }
      
    } else {
      
   # print(sortedCounts)
      humanConsensus = names(sortedCounts[1] )
      
    }
  }
  
  return(humanConsensus)
}


In [8]:
getSetSize = function(genesStr){
    geneList = str_split(string = genesStr, pattern = "[, ]")[[1]]
    return(length(geneList))
}

#### NeST

In [9]:
LLM_human_NeST = read_xlsx(path = 'data/NeST blinded.xlsx', sheet = 'Sheet1')


In [10]:
LLM_human_NeST = LLM_human_NeST %>% 
mutate(DataSet = "NeST") %>%
rowwise() %>%
mutate(humanConsensusMasked = getHumanConsensus(Winner_human1,Winner_human2,  Winner_human3, OverridingDecision )) %>%
mutate(humanConsensus = NeST_unmaskingDict[ humanConsensusMasked])

In [11]:
table(LLM_human_NeST$humanConsensus)


     GO     LLM Neither    Tied 
     13      18      13       6 

#### MSigDB

In [12]:
LLM_human_MSigDB = read_xlsx(path = 'data/MSigDB blinded.xlsx', sheet = 'Sheet1') 


In [13]:
dim(LLM_human_MSigDB)

[1] 50  6

In [14]:
LLM_human_MSigDB = LLM_human_MSigDB %>% 
mutate(DataSet = "MSigDB") %>%
rowwise() %>%
mutate(humanConsensusMasked = getHumanConsensus(Winner_human1,Winner_human2,  Winner_human3 )) %>%
mutate(humanConsensus = MSigDB_unmaskingDict[ humanConsensusMasked])

In [15]:
table(LLM_human_MSigDB$humanConsensus)


     GO     LLM Neither    Tied 
      7      24      10       9 

runVersion### Read in the semantic analyses

In [16]:
runVersion

[1] "test"

In [17]:
MSigDB_table_sub_LLM_GSEA_simVals_DF = read_delim("data/MSigDB_table_subset_LLM_GSEA_simVals_refs_DF.tsv", delim = "\t")


New names:
• `` -> `...1`
Rows: 3 Columns: 21
── Column specification ──────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (12): Name, Genes, Genes.1, LLM Name, LLM Analysis, Overlap, Genes.2, Ge...
dbl  (9): ...1, Unnamed: 0.1, index, Unnamed: 0, Rank, P-value, Adjusted P-v...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [18]:
if (runVersion == "test"){
    MSigDB_table_sub_LLM_GSEA_simVals_DF = read_delim("data/MSigDB_table_subset_LLM_GSEA_simVals_refs_DF.tsv", delim = "\t")
} else {
    MSigDB_table_sub_LLM_GSEA_simVals_DF = read_delim("data/MSigDB_table_LLM_GSEA_simVals_refs_DF.tsv", delim = "\t")
    }

New names:
• `` -> `...1`
Rows: 3 Columns: 21
── Column specification ──────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (12): Name, Genes, Genes.1, LLM Name, LLM Analysis, Overlap, Genes.2, Ge...
dbl  (9): ...1, Unnamed: 0.1, index, Unnamed: 0, Rank, P-value, Adjusted P-v...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [19]:
MSigDB_table_sub_LLM_GSEA_simVals_DF = rename(MSigDB_table_sub_LLM_GSEA_simVals_DF, 
                                               c("Human Name" ="Name",
                                                "GSEA genes" = "Genes.1"))

In [20]:
MSigDB_table_sub_LLM_GSEA_simVals_DF = MSigDB_table_sub_LLM_GSEA_simVals_DF %>%
rowwise() %>%
mutate(n_Genes = getSetSize(`Genes`),
      log2_n_Genes = log2(n_Genes))

#### NeST

In [21]:
if (runVersion == "test"){
    NeST_table_sub_LLM_GSEA_simVals_DF = read_delim("data/NeST_table_subset_LLM_GSEA_simVals_refs_DF.tsv", delim = "\t")
} else {
    NeST_table_sub_LLM_GSEA_simVals_DF = read_delim("data/NeST_table_LLM_GSEA_simVals_refs_DF.tsv", delim = "\t")
    }

New names:
• `` -> `...1`
Rows: 3 Columns: 22
── Column specification ──────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (13): NEST ID, name_new, Genes, Genes.1, LLM Name, LLM Analysis, Overlap...
dbl  (9): ...1, Unnamed: 0.1, index, Unnamed: 0, Rank, P-value, Adjusted P-v...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [22]:
NeST_table_sub_LLM_GSEA_simVals_DF

...1 Unnamed: 0.1 index Unnamed: 0 NEST ID  name_new                        
1 0    0            420   0          NEST:125 Extended mismatch repair complex
2 1    1            228   0          NEST:134 RPA complex associated proteins 
3 2    2              0   0          NEST:256 MAPK signaling                  
  Genes                                              
1 ERCC1,ERCC4,MLH1,MLH3,MSH2,MSH3,MSH6,PMS1,PMS2,SLX4
2 ATM,ATR,BLM,BRCA1,XPA,XPC                          
3 ERBB2,ERBB4,FGFR2,FGFR4,HRAS,KRAS                  
  Genes.1                            LLM Name                               
1 MSH6;MSH2;MSH3;PMS2;MLH1;MLH3;PMS1 DNA Mismatch Repair and Excision Repair
2 BLM;XPA;ATM;XPC;BRCA1;ATR          DNA Damage Response and Repair         
3 ERBB4;ERBB2;KRAS;FGFR4;HRAS;FGFR2  Receptor Tyrosine Kinase Signaling     
  LLM Analysis                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
1 The primary biological process performed by this system of interacting proteins is DNA mismatch repair and excision repair. These processes are crucial for maintaining genomic stability and preventing mutations.\n\n1. ERCC1 and ERCC4: These proteins form a heterodimeric endonuclease complex involved in nucleotide excision repair (NER). ERCC1 is responsible for DNA damage recognition, while ERCC4 provides the catalytic activity for DNA incision. Together, they remove DNA lesions caused by UV radiation and other genotoxic agents, preventing mutations and maintaining genomic stability.\n\n2. MLH1, MLH3, PMS1, and PMS2: These proteins are part of the MutL family and play essential roles in DNA mismatch repair (MMR). MLH1 forms heterodimers with PMS2 (MutLα), MLH3 (

In [23]:
NeST_table_sub_LLM_GSEA_simVals_DF = rename(NeST_table_sub_LLM_GSEA_simVals_DF, 
                                            c( "GSEA genes" = "Genes.1"))

In [24]:
NeST_table_sub_LLM_GSEA_simVals_DF = NeST_table_sub_LLM_GSEA_simVals_DF %>%
rowwise() %>%
mutate(n_Genes = getSetSize(`Genes`),
      log2_n_Genes = log2(n_Genes))

### Join the dataframe together

In [25]:
NeST_analysis_DF = inner_join(x = NeST_table_sub_LLM_GSEA_simVals_DF[,c("NEST ID", "Genes", "LLM Name", "LLM Analysis", "References",
                                      "GO term", "GO ID", "GSEA genes", "Overlap", "P-value", "Adjusted P-value", 
                                      "LLM_name_human_name_sim", "GO_term_human_name_sim", "winner", 
                                      "n_Genes", "log2_n_Genes")],
                              y = LLM_human_NeST[,c("NEST ID", "Human Name", "humanConsensus", "OverridingDecision", "DataSet")],
                              by = "NEST ID")
                                  

In [29]:
MSigDB_analysis_DF = inner_join(x = MSigDB_table_sub_LLM_GSEA_simVals_DF[,c("Human Name", "Genes", "LLM Name", "LLM Analysis", "References",
                                      "GO term", "GO ID", "GSEA genes", "Overlap", "P-value", "Adjusted P-value", 
                                      "LLM_name_human_name_sim", "GO_term_human_name_sim", "winner", 
                                      "n_Genes", "log2_n_Genes")],
                              y = LLM_human_MSigDB[,c("Human Name", "humanConsensus", "DataSet")],
                                 by =  "Human Name")
                                  

In [30]:
merged_analysis_DF = rbind.fill(NeST_analysis_DF,MSigDB_analysis_DF)

In [31]:
merged_analysis_DF = merged_analysis_DF %>%
rowwise() %>%
mutate(log2_rat_LLM_GO = log2(LLM_name_human_name_sim/GO_term_human_name_sim),
       diff_LLM_GO = LLM_name_human_name_sim - GO_term_human_name_sim)

In [32]:
merged_analysis_DF$humanConsensus <- factor(merged_analysis_DF$humanConsensus, levels=c("LLM", "GO", "Tied", "Neither"))


In [33]:
merged_analysis_DF$winner <- factor(merged_analysis_DF$winner, levels=c("LLM", "GO", "Tied", "Neither"))


In [34]:
if (runVersion == "test"){
    mergedFile = "data/merged_subset_LLM_GSEA_simVals_refs.txt"
} else {
    mergedFile = "data/merged_LLM_GSEA_simVals_refs.txt"
}

    
    


In [35]:
write_delim(x = merged_analysis_DF[,c('DataSet','Human Name', 'NEST ID', 'Genes', 'n_Genes', 'log2_n_Genes',
                                     'GO term','GO ID','GSEA genes', 'Overlap','P-value','Adjusted P-value',
                                     'LLM Name','LLM Analysis','References',
                                      'LLM_name_human_name_sim', 'GO_term_human_name_sim',
                                      'winner','humanConsensus', 'OverridingDecision')],
            file = mergedFile, delim = "\t")


In [36]:
merged_analysis_DF$DataSet

[1] "NeST"   "NeST"   "NeST"   "MSigDB" "MSigDB" "MSigDB"

### Get percentage of genes used in each of LLM and GSEA

In [37]:
library(comprehenr)


In [38]:
getGSEAGeneCount = function(systemGenes, GSEA_Genes){
    systemGenesList = str_split(string = systemGenes, pattern = " ")[[1]];
    GSEA_GenesList = str_split(string = GSEA_Genes, pattern = ";")[[1]];
    return(length(intersect(systemGenesList, GSEA_GenesList)))
    }
    

In [39]:
getLLMGeneCount  = function(GenesFixed, LLM_Analysis){
    systemGenesList = str_split(string = GenesFixed, pattern = " ")[[1]];
    
    n_LLMGenes = to_vec(for(systemGene in systemGenesList) str_detect(string = LLM_Analysis, pattern = systemGene)) %>%
           sum()
        
    return(n_LLMGenes)


    }

In [40]:
merged_analysis_DF = merged_analysis_DF %>% 
rowwise() %>%
mutate(GenesFixed = str_replace_all(string = `Genes`, pattern = ",", replacement = " "),
      n_overlap_GSEA = getGSEAGeneCount(`GenesFixed`, `GSEA genes`),
      perc_overlap_GSEA = 100*(n_overlap_GSEA/n_Genes),
      n_overlap_LLM = getLLMGeneCount(`GenesFixed`, `LLM Analysis`),
      perc_overlap_LLM = 100*(n_overlap_LLM/n_Genes))
  

In [42]:
write_delim(x = merged_analysis_DF[,c('DataSet','Human Name', 'NEST ID', 'GenesFixed', 'n_Genes', 'log2_n_Genes',
                                     'GO term','GO ID','GSEA genes', 'n_overlap_GSEA', 'perc_overlap_GSEA','P-value','Adjusted P-value',
                                     'LLM Name','LLM Analysis','n_overlap_LLM','perc_overlap_LLM', 'References',
                                      'LLM_name_human_name_sim', 'GO_term_human_name_sim',
                                      'winner','humanConsensus', 'OverridingDecision')],
            file = mergedFile, delim = "\t")